<a href="https://colab.research.google.com/github/saumyverma/AI/blob/main/Model-bert-base-uncased-fail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install torch --upgrade
!pip install pandas
!pip install wandb


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

# Load your dataset
import json
with open("qa_data.json", "r") as f:
    data = json.load(f)

# Convert to Hugging Face Dataset
qa_dataset = Dataset.from_list(data)

# Load tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize dataset
def preprocess(example):
    return tokenizer(
        example["question"],
        example["answer"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = qa_dataset.map(preprocess, batched=True)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_name)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments
import wandb
   wandb.login(key='2c531198b69178f2c81d9abbbe69b39f72bbf177')

training_args = TrainingArguments(
    output_dir="./results",          # Directory to save checkpoints
    eval_strategy="epoch",    # Evaluate at the end of each epoch
    learning_rate=2e-5,             # Learning rate
    per_device_train_batch_size=8,  # Batch size
    num_train_epochs=3,             # Number of epochs
    weight_decay=0.01,              # Regularization
    save_total_limit=2,             # Limit the number of saved checkpoints
    logging_dir="./logs",           # Directory for logs
)


In [ ]:
import pandas as pd

display(tokenized_dataset)
# Displaying the first 3 rows
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
     eval_dataset=tokenized_dataset,# Use the tokenized dataset
)

# Train the model
trainer.train()




Dataset({
    features: ['question', 'answer', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3
})

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 